<a href="https://colab.research.google.com/github/Matheus0697/LoginResponsive/blob/main/RecomendaFilmesPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# IDs das planilhas Google
sheet_id_filmes = '1QnKjOTBYDwItVJJiwlz46tBs5xR2EQyDKa6XJY9iD4o'
sheet_id_avaliacoes = '1ZOJCtiD1SAN9bLgVMgLSWPMqG6EoX5YCVWsLhD3SGow'

# URLs para acessar os dados das planilhas Google
url_filmes = f'https://docs.google.com/spreadsheets/d/{sheet_id_filmes}/export?format=csv'
url_avaliacoes = f'https://docs.google.com/spreadsheets/d/{sheet_id_avaliacoes}/export?format=csv'

# Carregando os dados das planilhas em DataFrames do Pandas
filmes = pd.read_csv(url_filmes)
avaliacoes = pd.read_csv(url_avaliacoes)

# Filtrando colunas relevantes e renomeando-as
filmes = filmes[['id', 'original_title', 'original_language', 'vote_count']]
filmes.columns = ['ID_FILME', 'TITULO', 'LINGUAGEM', 'QT_AVALIACOES']

avaliacoes = avaliacoes[['userId', 'movieId', 'rating']]
avaliacoes.columns = ['ID_USUARIO', 'ID_FILME', 'AVALIACAO']

# Removendo quaisquer valores nulos do DataFrame de filmes
filmes.dropna(inplace=True)

# Selecionando apenas os usuários que fizeram mais de 999 avaliações
usuarios_ativos = avaliacoes['ID_USUARIO'].value_counts() > 999
usuarios_ativos = usuarios_ativos[usuarios_ativos].index
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(usuarios_ativos)]

# Filtrando filmes que têm mais de 999 avaliações e que são em inglês
filmes = filmes[(filmes['QT_AVALIACOES'] > 999) & (filmes['LINGUAGEM'] == 'en')]

# Convertendo a coluna ID_FILME para o tipo inteiro
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

# Unindo os DataFrames de avaliações e filmes com base no ID_FILME
avaliacoes_e_filmes = avaliacoes.merge(filmes, on='ID_FILME')

# Removendo duplicatas no DataFrame unificado
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO', 'ID_FILME'], inplace=True)

# Removendo a coluna ID_FILME após a mesclagem
avaliacoes_e_filmes.drop(columns='ID_FILME', inplace=True)

# Criando uma tabela pivô com filmes como índices e usuários como colunas
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns='ID_USUARIO', index='TITULO', values='AVALIACAO').fillna(0)

# Convertendo a tabela pivô para uma matriz esparsa para economizar memória
filmes_sparse = csr_matrix(filmes_pivot)

# Treinando o modelo KNN (K-Nearest Neighbors) para recomendações
modelo = NearestNeighbors(algorithm='brute')
modelo.fit(filmes_sparse)

# Calculando a média das avaliações para cada filme
filmes_media_avaliacoes = avaliacoes_e_filmes.groupby('TITULO')['AVALIACAO'].mean()

# Selecionando os 10 filmes com a melhor média de avaliação
top_10_filmes = filmes_media_avaliacoes.sort_values(ascending=False).head(10)
top_10_filmes_df = top_10_filmes.reset_index()

# Arredondando as médias das avaliações para duas casas decimais
top_10_filmes_df['AVALIACAO'] = top_10_filmes_df['AVALIACAO'].round(2)

# Exibindo os 10 filmes mais bem avaliados
print("Top 10 filmes mais bem avaliados:")
print(top_10_filmes_df.to_string(index=False, header=True))

# Solicitando ao usuário que digite o nome de um filme para obter recomendações
filme_input = input("Digite o nome do filme para obter recomendações: ").strip().lower()

# Convertendo os títulos dos filmes para minúsculas para correspondência de strings
filmes_pivot_lower = filmes_pivot.copy()
filmes_pivot_lower.index = filmes_pivot_lower.index.str.lower()

# Fazendo recomendações com base no filme informado pelo usuário
if filme_input in filmes_pivot_lower.index:
    # Encontrando os vizinhos mais próximos do filme informado
    distances, suggestions = modelo.kneighbors(filmes_pivot_lower.loc[[filme_input]].values)

    # Exibindo os filmes recomendados, excluindo o filme de entrada
    print(f"Recomendações para '{filme_input}':")
    for i in range(len(suggestions[0])):
        recommended_movie = filmes_pivot.index[suggestions[0][i]]
        recommended_movie_lower = recommended_movie.lower()  # Converte para minúsculas
        if recommended_movie_lower != filme_input:  # Verifica se não é o filme de entrada
            print(recommended_movie)
else:
    # Exibindo mensagem de erro caso o filme não seja encontrado
    print("Filme não encontrado. Por favor, verifique o nome e tente novamente.")



<ipython-input-27-62e50f36494b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes.dropna(inplace=True)


Top 10 filmes mais bem avaliados:
                                       TITULO  AVALIACAO
                                       Zodiac       5.00
                            The Mummy Returns       5.00
                                         Hulk       5.00
                               The Green Mile       4.50
                                      Lawless       4.50
                                    Rush Hour       4.50
The Lord of the Rings: The Return of the King       4.50
                         The Bourne Supremacy       4.50
                             Dawn of the Dead       4.33
                                 Notting Hill       4.25
Digite o nome do filme para obter recomendações: superman
Recomendações para 'superman':
The Chronicles of Narnia: Prince Caspian
Ocean's Eleven
Mission: Impossible II
Memento
